In [2]:
import sys, os, _pickle as pickle
import tensorflow as tf
import numpy as np
import nltk
from sklearn.metrics import f1_score
import keras

Using TensorFlow backend.


In [3]:
data_dir = '../data'                        # Directory for Data and Other files
ckpt_dir = '../checkpoint'                  # Directory for Checkpoints 
word_embd_dir = '../checkpoint/word_embd'   # Directory for Checkpoints of Word Embedding Layer
# model_dir = '../checkpoint/modelv1'         # Directory for Checkpoints of Model
model_DIR = r"checkpoint/modelv1" # Directory for Checkpoints of Model

In [4]:
word_embd_dim = 100       # Dimension of embedding layer for words
pos_embd_dim = 25         # Dimension of embedding layer for POS Tags
dep_embd_dim = 25         # Dimension of embedding layer for Dependency Types

word_vocab_size = 400001  # Vocab size for Words
pos_vocab_size = 10       # Vocab size for POS Tags
dep_vocab_size = 21       # Vocab size for Dependency Types
word_state_size = 100
other_state_size = 100
relation_classes = 19     # No. of Relation Classes
state_size = 100          # Dimension of States of LSTM-RNNs
batch_size = 10           # Batch Size for training

channels = 100      # No. of types of features to feed in LSTM-RNN
lambda_l2 = 0.0001
max_len_path = 10         # Maximum length of sequence

In [5]:
# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()
grap = tf.Graph()

In [6]:
with grap.as_default():
    with tf.name_scope("input"):
    
    # Length of the sequence = 2X10
        path_length = tf.placeholder(tf.int32, shape=[2, batch_size], name="path1_length") 

        # Words in the sequence  = 2X10X10
        word_ids = tf.placeholder(tf.int32, shape=[2, batch_size, max_len_path], name="word_ids") 

         # POS Tags in the sequence = 2X10X10
        pos_ids = tf.placeholder(tf.int32, [2, batch_size, max_len_path], name="pos_ids") 

        # Dependency Types in the sequence = 2X10X10
        dep_ids = tf.placeholder(tf.int32, [2, batch_size, max_len_path], name="dep_ids") 

         # True Relation btw the entities = [10]
        y = tf.placeholder(tf.int32, [batch_size], name="y")   
        init=tf.global_variables_initializer()   # Initializing all Variables within the scope

In [ ]:
# word_ids, dep_ids

In [7]:
# Embedding Layer of Words 
with grap.as_default():
    with tf.name_scope("word_embedding"):
        W = tf.Variable(tf.constant(0.0, shape=[word_vocab_size, word_embd_dim]), name="W")
        embedding_placeholder = tf.placeholder(tf.float32,[word_vocab_size, word_embd_dim])
        embedding_init = W.assign(embedding_placeholder)
        embedded_word = tf.nn.embedding_lookup(W, word_ids)
        word_embedding_saver = tf.train.Saver({"word_embedding/W": W})
        init=tf.global_variables_initializer()   # Initializing all Variables within the scope

    # Embedding Layer of POS Tags 
    with tf.name_scope("pos_embedding"):
        W = tf.Variable(tf.random_uniform([pos_vocab_size, pos_embd_dim]), name="W")
        embedded_pos = tf.nn.embedding_lookup(W, pos_ids)
        pos_embedding_saver = tf.train.Saver({"pos_embedding/W": W})
        init=tf.global_variables_initializer()   # Initializing all Variables within the scope

    # Embedding Layer of Dependency Types 
    with tf.name_scope("dep_embedding"):
        W = tf.Variable(tf.random_uniform([dep_vocab_size, dep_embd_dim]), name="W")
        embedded_dep = tf.nn.embedding_lookup(W, dep_ids)
        dep_embedding_saver = tf.train.Saver({"dep_embedding/W": W})
        init=tf.global_variables_initializer()   # Initializing all Variables within the scope

In [ ]:
with grap.as_default():
    my_scope = 'pos_embedding'
    scope_variables=  tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope=my_scope )
    print(scope_variables)

In [ ]:
# pos_embedding_saver
# embedded_word[0], embedded_pos, embedded_dep
# type(embedded_word)

### Creating LSTM Layers for Different Embeddings

In [8]:
word_sz_sequence = 10
n_units = 100
n_layers = 3
channels_word=100
tf.reset_default_graph()

# with grap.as_default():
#  For Word Embeddings
with grap.as_default():
    with tf.variable_scope("word_lstm1"):
        inputs = tf.keras.layers.Input(batch_shape=(batch_size, word_sz_sequence, channels_word))
        cells = [tf.keras.layers.GRUCell(n_units) for _ in range(n_layers)]
        state_series_w1 = tf.keras.layers.RNN(cells, stateful=True, return_sequences=True, return_state=False)(embedded_word[0])
        state_series_word1 = tf.reduce_max(state_series_w1, axis=1)
        init=tf.global_variables_initializer()   # Initializing all Variables within the scope
        print("state_series_w1", state_series_w1)
        print("state_series_word1", state_series_word1)
    #  For Word Embeddings 2
    with tf.variable_scope("word_lstm2"):
        inputs = tf.keras.layers.Input(batch_shape=(batch_size, word_sz_sequence, channels_word))
        cells = [tf.keras.layers.GRUCell(n_units) for _ in range(n_layers)]
        state_series_w2 = tf.keras.layers.RNN(cells, stateful=True, return_sequences=True, return_state=False)(embedded_word[1])
        state_series_word2 = tf.reduce_max(state_series_w1, axis=1)
        init=tf.global_variables_initializer()   # Initializing all Variables within the scope
        print("state_series_w2", state_series_w2)
        print("state_series_word2", state_series_word2)
    

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
state_series_w1 Tensor("word_lstm1/rnn/transpose_1:0", shape=(10, 10, 100), dtype=float32)
state_series_word1 Tensor("word_lstm1/Max:0", shape=(10, 100), dtype=float32)
state_series_w2 Tensor("word_lstm2/rnn_1/transpose_1:0", shape=(10, 10, 100), dtype=float32)
state_series_word2 Tensor("word_lstm2/Max:0", shape=(10, 100), dtype=float32)


In [9]:
#  For POS Embeddings 1
pos_sz_sequence_POS = 10
channels_POS = 25
n_units_POS = 25
with grap.as_default():
    with tf.variable_scope("pos_lstm1"):
        inputs = tf.keras.layers.Input(batch_shape=(batch_size, pos_sz_sequence_POS, channels_POS)) # 10X10X25
        cells = [tf.keras.layers.GRUCell(n_units_POS) for _ in range(n_layers)]
        state_series_p1 = tf.keras.layers.RNN(cells, stateful=True, return_sequences=True, return_state=False)(embedded_pos[0])
        state_series_pos1 = tf.reduce_max(state_series_p1, axis=1)
        init=tf.global_variables_initializer()   # Initializing all Variables within the scope
        print("state_series_p1", state_series_p1)
        print("state_series_pos1", state_series_pos1)

    #  For POS Embeddings 2
    with tf.variable_scope("pos_lstm2"):
        inputs = tf.keras.layers.Input(batch_shape=(batch_size, pos_sz_sequence_POS, channels_POS)) # 10X25X3
        cells = [tf.keras.layers.GRUCell(n_units_POS) for _ in range(n_layers)]
        state_series_p2 = tf.keras.layers.RNN(cells, stateful=True, return_sequences=True, return_state=False)(embedded_pos[1])
        state_series_pos2 = tf.reduce_max(state_series_p2, axis=1)
        init=tf.global_variables_initializer()   # Initializing all Variables within the scope
        print("state_series_w2", state_series_p2)
        print("state_series_pos2", state_series_pos2)

state_series_p1 Tensor("pos_lstm1/rnn_2/transpose_1:0", shape=(10, 10, 25), dtype=float32)
state_series_pos1 Tensor("pos_lstm1/Max:0", shape=(10, 25), dtype=float32)
state_series_w2 Tensor("pos_lstm2/rnn_3/transpose_1:0", shape=(10, 10, 25), dtype=float32)
state_series_pos2 Tensor("pos_lstm2/Max:0", shape=(10, 25), dtype=float32)


In [ ]:
with grap.as_default():
    my_scope = 'pos_lstm2'
    scope_variables=  tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope=my_scope )
    print(scope_variables)

In [10]:
#  For POS Embeddings 2
dep_sz_sequence = 10
channels_DEP = 25
n_units_DEP = 25
with grap.as_default():
    with tf.variable_scope("dep_lstm1"):
        inputs = tf.keras.layers.Input(batch_shape=(batch_size, dep_sz_sequence, channels_DEP))
        cells = [tf.keras.layers.GRUCell(n_units_DEP) for _ in range(n_layers)]
        state_series_d1 = tf.keras.layers.RNN(cells, stateful=True, return_sequences=True, return_state=False)(embedded_dep[0])
        state_series_dep1 = tf.reduce_max(state_series_d1, axis=1)
        init=tf.global_variables_initializer()   # Initializing all Variables within the scope
        print("state_series_d1", state_series_d1)
        print("state_series_dep1", state_series_dep1)

    with tf.variable_scope("dep_lstm2"):
        inputs = tf.keras.layers.Input(batch_shape=(batch_size, dep_sz_sequence, channels_DEP))#10X10X25
        cells = [tf.keras.layers.GRUCell(n_units_DEP) for _ in range(n_layers)]
        state_series_d2 = tf.keras.layers.RNN(cells, stateful=True, return_sequences=True, return_state=False)(embedded_dep[1])
        state_series_dep2 = tf.reduce_max(state_series_d2, axis=1)
        init=tf.global_variables_initializer()   # Initializing all Variables within the scope
        print("state_series_d2", state_series_d2)
        print("state_series_dep2", state_series_dep2)


state_series_d1 Tensor("dep_lstm1/rnn_4/transpose_1:0", shape=(10, 10, 25), dtype=float32)
state_series_dep1 Tensor("dep_lstm1/Max:0", shape=(10, 25), dtype=float32)
state_series_d2 Tensor("dep_lstm2/rnn_5/transpose_1:0", shape=(10, 10, 25), dtype=float32)
state_series_dep2 Tensor("dep_lstm2/Max:0", shape=(10, 25), dtype=float32)


In [11]:
with grap.as_default():
    state_series1 = tf.concat([state_series_word1, state_series_pos1, state_series_dep1], 1)
    state_series2 = tf.concat([state_series_word2, state_series_pos2, state_series_dep2], 1)

    state_series = tf.concat([state_series1, state_series2], 1)
    init=tf.global_variables_initializer()   # Initializing all Variables within the scope
    print("state_series", state_series)

state_series Tensor("concat_2:0", shape=(10, 300), dtype=float32)


In [12]:
# grap = tf.Graph()
with grap.as_default():
    with tf.name_scope("hidden_layer"):
        W = tf.Variable(tf.truncated_normal([300, 100], -0.1, 0.1), name="W")
#         b = tf.Variable(tf.zeros([100]), name="b")
        b = tf.Variable(tf.zeros([100]), name="b")
        y_hidden_layer = tf.matmul(state_series, W) + b
        init=tf.global_variables_initializer()   # Initializing all Variables within the scope

    with tf.name_scope("softmax_layer"):
        W = tf.Variable(tf.truncated_normal([100, relation_classes], -0.1, 0.1), name="W")
        b = tf.Variable(tf.zeros([relation_classes]), name="b")
        logits = tf.matmul(y_hidden_layer, W) + b
        predictions = tf.argmax(logits, 1)
        init=tf.global_variables_initializer()   # Initializing all Variables within the scope

print(logits)

Tensor("softmax_layer/add:0", shape=(10, 19), dtype=float32)


In [ ]:
with grap.as_default():
    my_scope = 'softmax_layer'
    scope_variables=  tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope= my_scope )
    for scope_var in scope_variables:
        print(scope_var.name)

In [13]:
with grap.as_default():
    tv_all = tf.trainable_variables()
    tv_regu = []
    non_reg = ["word_embedding/W:0","pos_embedding/W:0",'dep_embedding/W:0',"global_step:0",'hidden_layer/b:0','softmax_layer/b:0']
    for t in tv_all:
        if t.name not in non_reg:
#             if(t.name.find('biases')==-1):
            if(t.name.find('bias')==-1):
                tv_regu.append(t)

In [ ]:
# for tv in tv_regu:
#     print(tv.name)

In [14]:
with grap.as_default():
    with tf.name_scope("loss"):
        l2_loss = lambda_l2 * tf.reduce_sum([ tf.nn.l2_loss(v) for v in tv_regu ])
        loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y))
        total_loss = loss + l2_loss
        init=tf.global_variables_initializer()   # Initializing all Variables within the scope

    global_step = tf.Variable(0, name="global_step")

    optimizer = tf.train.AdamOptimizer(0.001).minimize(total_loss, global_step=global_step)
    init=tf.global_variables_initializer()  # Initializing the optimizer variables

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [14]:
### y, logits
# tv_regu
print(tv_all)

[<tf.Variable 'word_embedding/W:0' shape=(400001, 100) dtype=float32_ref>, <tf.Variable 'pos_embedding/W:0' shape=(10, 25) dtype=float32_ref>, <tf.Variable 'dep_embedding/W:0' shape=(21, 25) dtype=float32_ref>, <tf.Variable 'word_lstm1/rnn/kernel:0' shape=(100, 300) dtype=float32>, <tf.Variable 'word_lstm1/rnn/recurrent_kernel:0' shape=(100, 300) dtype=float32>, <tf.Variable 'word_lstm1/rnn/bias:0' shape=(300,) dtype=float32>, <tf.Variable 'word_lstm1/rnn/kernel_1:0' shape=(100, 300) dtype=float32>, <tf.Variable 'word_lstm1/rnn/recurrent_kernel_1:0' shape=(100, 300) dtype=float32>, <tf.Variable 'word_lstm1/rnn/bias_1:0' shape=(300,) dtype=float32>, <tf.Variable 'word_lstm1/rnn/kernel_2:0' shape=(100, 300) dtype=float32>, <tf.Variable 'word_lstm1/rnn/recurrent_kernel_2:0' shape=(100, 300) dtype=float32>, <tf.Variable 'word_lstm1/rnn/bias_2:0' shape=(300,) dtype=float32>, <tf.Variable 'word_lstm1/rnn/Variable:0' shape=(10, 100) dtype=float32>, <tf.Variable 'word_lstm1/rnn/Variable_1:0' s

In [15]:
f = open(data_dir + '/vocab.pkl', 'rb')
vocab = pickle.load(f)
f.close()

word2id = dict((w, i) for i,w in enumerate(vocab))
id2word = dict((i, w) for i,w in enumerate(vocab))

unknown_token = "UNKNOWN_TOKEN"
word2id[unknown_token] = word_vocab_size -1
id2word[word_vocab_size-1] = unknown_token

pos_tags_vocab = []
for line in open(data_dir + '/pos_tags.txt'):
        pos_tags_vocab.append(line.strip())

dep_vocab = []
for line in open(data_dir + '/dependency_types.txt'):
    dep_vocab.append(line.strip())

relation_vocab = []
for line in open(data_dir + '/relation_types.txt'):
    relation_vocab.append(line.strip())


rel2id = dict((w, i) for i,w in enumerate(relation_vocab))
id2rel = dict((i, w) for i,w in enumerate(relation_vocab))

pos_tag2id = dict((w, i) for i,w in enumerate(pos_tags_vocab))
id2pos_tag = dict((i, w) for i,w in enumerate(pos_tags_vocab))

dep2id = dict((w, i) for i,w in enumerate(dep_vocab))
id2dep = dict((i, w) for i,w in enumerate(dep_vocab))

pos_tag2id['OTH'] = 9
id2pos_tag[9] = 'OTH'

dep2id['OTH'] = 20
id2dep[20] = 'OTH'

JJ_pos_tags = ['JJ', 'JJR', 'JJS']
NN_pos_tags = ['NN', 'NNS', 'NNP', 'NNPS']
RB_pos_tags = ['RB', 'RBR', 'RBS']
PRP_pos_tags = ['PRP', 'PRP$']
VB_pos_tags = ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']
_pos_tags = ['CC', 'CD', 'DT', 'IN']

def pos_tag(x):
    if x in JJ_pos_tags:
        return pos_tag2id['JJ']
    if x in NN_pos_tags:
        return pos_tag2id['NN']
    if x in RB_pos_tags:
        return pos_tag2id['RB']
    if x in PRP_pos_tags:
        return pos_tag2id['PRP']
    if x in VB_pos_tags:
        return pos_tag2id['VB']
    if x in _pos_tags:
        return pos_tag2id[x]
    else:
        return 9

In [16]:
f = open(data_dir + '/train_paths', 'rb')
word_p1, word_p2, dep_p1, dep_p2, pos_p1, pos_p2 = pickle.load(f)
f.close()

relations = []
for line in open(data_dir + '/train_relations.txt'):
#     print(line.strip().split())
    relations.append(line.strip().split()[0])
print(relations)
# rel2id

['Component-Whole(e2,e1)', 'Other', 'Instrument-Agency(e2,e1)', 'Other', 'Member-Collection(e1,e2)', 'Other', 'Cause-Effect(e2,e1)', 'Entity-Destination(e1,e2)', 'Content-Container(e1,e2)', 'Entity-Destination(e1,e2)', 'Member-Collection(e1,e2)', 'Other', 'Message-Topic(e1,e2)', 'Cause-Effect(e2,e1)', 'Instrument-Agency(e2,e1)', 'Message-Topic(e1,e2)', 'Instrument-Agency(e2,e1)', 'Product-Producer(e2,e1)', 'Component-Whole(e2,e1)', 'Member-Collection(e2,e1)', 'Entity-Origin(e1,e2)', 'Member-Collection(e2,e1)', 'Cause-Effect(e1,e2)', 'Other', 'Member-Collection(e2,e1)', 'Other', 'Cause-Effect(e1,e2)', 'Message-Topic(e1,e2)', 'Message-Topic(e1,e2)', 'Component-Whole(e1,e2)', 'Message-Topic(e2,e1)', 'Cause-Effect(e2,e1)', 'Product-Producer(e1,e2)', 'Entity-Destination(e1,e2)', 'Component-Whole(e1,e2)', 'Entity-Origin(e1,e2)', 'Other', 'Component-Whole(e2,e1)', 'Cause-Effect(e1,e2)', 'Instrument-Agency(e2,e1)']


In [17]:
rel2id

{'Other': 0,
 'Entity-Destination(e1,e2)': 1,
 'Cause-Effect(e2,e1)': 2,
 'Member-Collection(e2,e1)': 3,
 'Entity-Origin(e1,e2)': 4,
 'Message-Topic(e1,e2)': 5,
 'Component-Whole(e2,e1)': 6,
 'Component-Whole(e1,e2)': 7,
 'Instrument-Agency(e2,e1)': 8,
 'Product-Producer(e2,e1)': 9,
 'Content-Container(e1,e2)': 10,
 'Cause-Effect(e1,e2)': 11,
 'Product-Producer(e1,e2)': 12,
 'Content-Container(e2,e1)': 13,
 'Entity-Origin(e2,e1)': 14,
 'Message-Topic(e2,e1)': 15,
 'Instrument-Agency(e1,e2)': 16,
 'Member-Collection(e1,e2)': 17,
 'Entity-Destination(e2,e1)': 18}

In [ ]:
# word_p1
# max_len_path
# word_p1_ids.shape

In [18]:
length = len(word_p1)
num_batches = int(length/batch_size)

for i in range(length):
    for j, word in enumerate(word_p1[i]):
        word = word.lower()
        word_p1[i][j] = word if word in word2id else unknown_token 
    for k, word in enumerate(word_p2[i]):
        word = word.lower()
        word_p2[i][k] = word if word in word2id else unknown_token 
    for l, d in enumerate(dep_p1[i]):
        dep_p1[i][l] = d if d in dep2id else 'OTH'
    for m, d in enumerate(dep_p2[i]):
        dep_p2[i][m] = d if d in dep2id else 'OTH'

word_p1_ids = np.ones([length, max_len_path],dtype=int)
word_p2_ids = np.ones([length, max_len_path],dtype=int)
pos_p1_ids = np.ones([length, max_len_path],dtype=int)
pos_p2_ids = np.ones([length, max_len_path],dtype=int)
dep_p1_ids = np.ones([length, max_len_path],dtype=int)
dep_p2_ids = np.ones([length, max_len_path],dtype=int)
rel_ids = np.array([rel2id[rel] for rel in relations])
path1_len = np.array([len(w) for w in word_p1], dtype=int)
path2_len = np.array([len(w) for w in word_p2])

for i in range(length):
    for j, w in enumerate(word_p1[i]):
        word_p1_ids[i][j] = word2id[w]
    for j, w in enumerate(word_p2[i]):
        word_p2_ids[i][j] = word2id[w]
    for j, w in enumerate(pos_p1[i]):
        pos_p1_ids[i][j] = pos_tag(w)
    for j, w in enumerate(pos_p2[i]):
        pos_p2_ids[i][j] = pos_tag(w)
    for j, w in enumerate(dep_p1[i]):
        dep_p1_ids[i][j] = dep2id[w]
    for j, w in enumerate(dep_p2[i]):
        dep_p2_ids[i][j] = dep2id[w]

# Model Loading and Prediction

In [31]:
os.getcwd()
Load_Model_DIR = r"/checkpoint/modelv1/"
Load_Model_DIR

'/checkpoint/modelv1/'

In [36]:
aas=2397972
print("{}.meta".format(aas))

2397972.meta


In [57]:
DIR = r"E:\The University of Texas at Dallas\Fall 2020\Natural Language Processing- CS 6320.501\NLP Project\Relation-Classification-using-Bidirectional-LSTM-Tree\Relation-Classification-using-Bidirectional-LSTM\LCA Shortest Path\checkpoint\modelv1"
checkpoint_file = tf.train.latest_checkpoint(DIR)
checkpoint_file

'E:\\The University of Texas at Dallas\\Fall 2020\\Natural Language Processing- CS 6320.501\\NLP Project\\Relation-Classification-using-Bidirectional-LSTM-Tree\\Relation-Classification-using-Bidirectional-LSTM\\LCA Shortest Path\\checkpoint\\modelv1\\-440'

In [56]:
# Load_Model_DIR = r"/checkpoint/modelv1/"
with grap.as_default():
    with tf.Session() as sess:
        checkpoint_file = tf.train.latest_checkpoint(DIR)
        saver = tf.train.import_meta_graph("{}.meta".format(checkpoint_file))
        saver.restore(sess, checkpoint_file)


INFO:tensorflow:Restoring parameters from E:\The University of Texas at Dallas\Fall 2020\Natural Language Processing- CS 6320.501\NLP Project\Relation-Classification-using-Bidirectional-LSTM-Tree\Relation-Classification-using-Bidirectional-LSTM\LCA Shortest Path\checkpoint\modelv1\-440


In [25]:
# saver
#     sess.run(tf.global_variables_initializer())
#     saver = tf.train.Saver(defer_build=True)

In [ ]:
# with grap.as_default():
#     print(tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='dep_embedding'))

In [ ]:
# f = open('data/word_embedding', 'rb')
# word_embedding = pickle.load(f)
# f.close()
# sess.run(embedding_init, feed_dict={embedding_placeholder:word_embedding})
# word_embedding_saver.save(sess, word_embd_dir + '/word_embd')
with grap.as_default():
    for i in tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='dep_embedding'):
        print(i.name)

In [ ]:
# model_dir = r"E:\The University of Texas at Dallas\Fall 2020\Natural Language Processing- CS 6320.501\NLP Project\Relation-Classification-using-Bidirectional-LSTM-Tree\Relation-Classification-using-Bidirectional-LSTM\checkpoint\modelv1"
model = tf.train.latest_checkpoint(model_DIR)
saver.restore(sess, model)

In [ ]:
# with grap.as_default():
#     glob_var=  tf.get_collection(tf.GraphKeys.VARIABLES)
# glob_var       

In [19]:
tf.train.latest_checkpoint(model_DIR)

In [19]:
# tf.convert_to_tensor(word_dict)
# word_dict
# grap
import numpy as np
from tensorflow.python.layers import base
import tensorflow as tf
import tensorflow.contrib.slim as slim
def model_summary():
    model_vars = tf.trainable_variables()
    slim.model_analyzer.analyze_vars(model_vars, print_info=True)

with grap.as_default():
    model_summary()

---------
Variables: name (type shape) [size]
---------
word_embedding/W:0 (float32_ref 400001x100) [40000100, bytes: 160000400]
pos_embedding/W:0 (float32_ref 10x25) [250, bytes: 1000]
dep_embedding/W:0 (float32_ref 21x25) [525, bytes: 2100]
word_lstm1/rnn/kernel:0 (float32 100x300) [30000, bytes: 120000]
word_lstm1/rnn/recurrent_kernel:0 (float32 100x300) [30000, bytes: 120000]
word_lstm1/rnn/bias:0 (float32 300) [300, bytes: 1200]
word_lstm1/rnn/kernel_1:0 (float32 100x300) [30000, bytes: 120000]
word_lstm1/rnn/recurrent_kernel_1:0 (float32 100x300) [30000, bytes: 120000]
word_lstm1/rnn/bias_1:0 (float32 300) [300, bytes: 1200]
word_lstm1/rnn/kernel_2:0 (float32 100x300) [30000, bytes: 120000]
word_lstm1/rnn/recurrent_kernel_2:0 (float32 100x300) [30000, bytes: 120000]
word_lstm1/rnn/bias_2:0 (float32 300) [300, bytes: 1200]
word_lstm1/rnn/Variable:0 (float32 10x100) [1000, bytes: 4000]
word_lstm1/rnn/Variable_1:0 (float32 10x100) [1000, bytes: 4000]
word_lstm1/rnn/Variable_2:0 (flo

In [21]:
with grap.as_default():
        train_vars= tf.trainable_variables()
        all_vars = tf.global_variables()
len(all_vars), len(train_vars)

(234, 80)

In [ ]:
all_vars

In [ ]:
train_vars

In [ ]:
# with grap.as_default():
#     with tf.Session() as sess:
#         print(tf.global_variables())

In [ ]:
# path_dict
# type(y_dict)
# word_dict[0].shape
# dep_dict[0].shape
# path_dict[0].shape

In [ ]:
# relations
# sess = tf.Session()
# sess.run(tf.local_variables_initializer())

In [ ]:
# init = tf.global_variables_initializer()
# with tf.Session() as sess:
#     sess.run(init)

In [ ]:
# tf.global_variables_initializer()
# init_op = tf.initialize_all_variables()

# sess = tf.Session()
# sess.run(init_op)
# with grap.as_default():
#     init_g = tf.global_variables_initializer()
#     init_l = tf.local_variables_initializer()
#     with tf.Session() as sess:
#         sess.run(init_g)
#         sess.run(init_l)

In [ ]:
# with tf.Session() as sess:
#     tf.initialize_all_variables().run()

In [ ]:
# Add an op to initialize the variables.
# init_op = tf.initialize_all_variables()

# Later, when launching the model
# with tf.Session() as sess:
#     print(sess.run(init_op))
  # Run the init operation


In [ ]:
with grap.as_default():
    keras.backend.get_session().run(tf.global_variables_initializer())

In [ ]:
# with grap.as_default():
tf.global_variables_initializer()

# Main Training 

In [38]:
import os
os.getcwd()

'E:\\The University of Texas at Dallas\\Fall 2020\\Natural Language Processing- CS 6320.501\\NLP Project\\Relation-Classification-using-Bidirectional-LSTM-Tree\\Relation-Classification-using-Bidirectional-LSTM\\LCA Shortest Path'

In [ ]:
# model_dir + '/model'
# model_DIR = r"checkpoint/modelv1"

In [34]:
num_epochs = 100
for i in range(num_epochs):
    for j in range(num_batches):
        path_dict = [path1_len[j*batch_size:(j+1)*batch_size], path2_len[j*batch_size:(j+1)*batch_size]]
        word_dict = [word_p1_ids[j*batch_size:(j+1)*batch_size], word_p2_ids[j*batch_size:(j+1)*batch_size]]
        pos_dict = [pos_p1_ids[j*batch_size:(j+1)*batch_size], pos_p2_ids[j*batch_size:(j+1)*batch_size]]
        dep_dict = [dep_p1_ids[j*batch_size:(j+1)*batch_size], dep_p2_ids[j*batch_size:(j+1)*batch_size]]
        y_dict = rel_ids[j*batch_size:(j+1)*batch_size]
        
#         path_dict = tf.convert_to_tensor(path_dict)
#         word_dict = tf.convert_to_tensor(word_dict)
#         pos_dict = tf.convert_to_tensor(pos_dict)
#         dep_dict = tf.convert_to_tensor(dep_dict)
#         y_dict = tf.convert_to_tensor(y_dict)
        
        feed_dict = {
            path_length:path_dict,
            word_ids:word_dict,
            pos_ids:pos_dict,
            dep_ids:dep_dict,
            y:y_dict}
        with grap.as_default():
#             with tf.Session(graph=grap) as sess:
            print("Sess Running")
            _, loss, step = sess.run([optimizer, total_loss, global_step], feed_dict)
            if step%10==0:
                print("Step:", step, "loss:",loss)
            if step % 20 == 0:
                saver.save(sess, model_DIR , global_step=step)
                print("Saved Model at", model_DIR)

Sess Running
Sess Running
Sess Running
Sess Running
Sess Running
Sess Running
Sess Running
Sess Running
Sess Running
Sess Running
Step: 50 loss: 1.3380758
Sess Running
Sess Running
Sess Running
Sess Running
Sess Running
Sess Running
Sess Running
Sess Running
Sess Running
Sess Running
Step: 60 loss: 1.461537
Saved Model at checkpoint/modelv1/
Sess Running
Sess Running
Sess Running
Sess Running
Sess Running
Sess Running
Sess Running
Sess Running
Sess Running
Sess Running
Step: 70 loss: 0.8698428
Sess Running
Sess Running
Sess Running
Sess Running
Sess Running
Sess Running
Sess Running
Sess Running
Sess Running
Sess Running
Step: 80 loss: 0.9936742
Saved Model at checkpoint/modelv1/
Sess Running
Sess Running
Sess Running
Sess Running
Sess Running
Sess Running
Sess Running
Sess Running
Sess Running
Sess Running
Step: 90 loss: 0.64139134
Sess Running
Sess Running
Sess Running
Sess Running
Sess Running
Sess Running
Sess Running
Sess Running
Sess Running
Sess Running
Step: 100 loss: 0.6554001

In [32]:
model_DIR = r"checkpoint/modelv1/"
model_DIR

'checkpoint/modelv1/'

In [ ]:
# with grap.as_default():
#         if i % 5 == 0:
#             saver.save(sess, model_dir + '/model')
#             print("Saved Model")

In [35]:
step

440

## Prediction 

In [36]:
# training accuracy
all_predictions = []
for j in range(num_batches):
    path_dict = [path1_len[j*batch_size:(j+1)*batch_size], path2_len[j*batch_size:(j+1)*batch_size]]
    word_dict = [word_p1_ids[j*batch_size:(j+1)*batch_size], word_p2_ids[j*batch_size:(j+1)*batch_size]]
    pos_dict = [pos_p1_ids[j*batch_size:(j+1)*batch_size], pos_p2_ids[j*batch_size:(j+1)*batch_size]]
    dep_dict = [dep_p1_ids[j*batch_size:(j+1)*batch_size], dep_p2_ids[j*batch_size:(j+1)*batch_size]]
    y_dict = rel_ids[j*batch_size:(j+1)*batch_size]

    feed_dict = {
        path_length:path_dict,
        word_ids:word_dict,
        pos_ids:pos_dict,
        dep_ids:dep_dict,
        y:y_dict}
    with grap.as_default():
        batch_predictions = sess.run(predictions, feed_dict)
        print("Sess Running for predictions")
        all_predictions.append(batch_predictions)

y_pred = []
for i in range(num_batches):
    for pred in all_predictions[i]:
        y_pred.append(pred)

count = 0
for i in range(batch_size*num_batches):
    count += y_pred[i]==rel_ids[i]
accuracy = count/(batch_size*num_batches) * 100

print("training accuracy", accuracy)

Sess Running for predictions
Sess Running for predictions
Sess Running for predictions
Sess Running for predictions
training accuracy 100.0


# Prediction on Testing dataset

In [20]:
f = open(data_dir + '/test_pathsv1', 'rb')
word_p1, word_p2, dep_p1, dep_p2, pos_p1, pos_p2 = pickle.load(f)
f.close()

relations = []
for line in open(data_dir + '/test_relations.txt'):
    relations.append(line.strip().split()[0])

length = len(word_p1)
num_batches = int(length/batch_size)

for i in range(length):
    for j, word in enumerate(word_p1[i]):
        word = word.lower()
        word_p1[i][j] = word if word in word2id else unknown_token 
    for k, word in enumerate(word_p2[i]):
        word = word.lower()
        word_p2[i][k] = word if word in word2id else unknown_token 
    for l, d in enumerate(dep_p1[i]):
        dep_p1[i][l] = d if d in dep2id else 'OTH'
    for m, d in enumerate(dep_p2[i]):
        dep_p2[i][m] = d if d in dep2id else 'OTH'

word_p1_ids = np.ones([length, max_len_path],dtype=int)
word_p2_ids = np.ones([length, max_len_path],dtype=int)
pos_p1_ids = np.ones([length, max_len_path],dtype=int)
pos_p2_ids = np.ones([length, max_len_path],dtype=int)
dep_p1_ids = np.ones([length, max_len_path],dtype=int)
dep_p2_ids = np.ones([length, max_len_path],dtype=int)
rel_ids = np.array([rel2id[rel] for rel in relations])
path1_len = np.array([len(w) for w in word_p1], dtype=int)
path2_len = np.array([len(w) for w in word_p2])

for i in range(length):
    for j, w in enumerate(word_p1[i]):
        word_p1_ids[i][j] = word2id[w]
    for j, w in enumerate(word_p2[i]):
        word_p2_ids[i][j] = word2id[w]
    for j, w in enumerate(pos_p1[i]):
        pos_p1_ids[i][j] = pos_tag(w)
    for j, w in enumerate(pos_p2[i]):
        pos_p2_ids[i][j] = pos_tag(w)
    for j, w in enumerate(dep_p1[i]):
        dep_p1_ids[i][j] = dep2id[w]
    for j, w in enumerate(dep_p2[i]):
        dep_p2_ids[i][j] = dep2id[w]

In [41]:
# word_p1

In [47]:
rel_ids

array([ 6,  0,  8, ..., 16, 13,  6])

In [48]:
# y_dict
len(dep_p1_ids)

2717

In [59]:
# test 
with grap.as_default():
    with tf.Session() as sess1:
        checkpoint_file = tf.train.latest_checkpoint(DIR)
        saver = tf.train.import_meta_graph("{}.meta".format(checkpoint_file))
        saver.restore(sess1, checkpoint_file)
        all_predictions = []
        for j in range(num_batches):
            path_dict = [path1_len[j*batch_size:(j+1)*batch_size], path2_len[j*batch_size:(j+1)*batch_size]]
            word_dict = [word_p1_ids[j*batch_size:(j+1)*batch_size], word_p2_ids[j*batch_size:(j+1)*batch_size]]
            pos_dict = [pos_p1_ids[j*batch_size:(j+1)*batch_size], pos_p2_ids[j*batch_size:(j+1)*batch_size]]
            dep_dict = [dep_p1_ids[j*batch_size:(j+1)*batch_size], dep_p2_ids[j*batch_size:(j+1)*batch_size]]
            y_dict = rel_ids[j*batch_size:(j+1)*batch_size]

            feed_dict = {
                path_length:path_dict,
                word_ids:word_dict,
                pos_ids:pos_dict,
                dep_ids:dep_dict,
                y:y_dict}
#             with grap.as_default():
#                 #     with tf.Session() as sess:
            batch_predictions = sess1.run(predictions, feed_dict)
            all_predictions.append(batch_predictions)

        y_pred = []
        for i in range(num_batches):
            for pred in all_predictions[i]:
                y_pred.append(pred)

        count = 0
        for i in range(batch_size*num_batches):
            count += y_pred[i]==rel_ids[i]
        accuracy = count/(batch_size*num_batches) * 100

        print("test accuracy", accuracy)

INFO:tensorflow:Restoring parameters from E:\The University of Texas at Dallas\Fall 2020\Natural Language Processing- CS 6320.501\NLP Project\Relation-Classification-using-Bidirectional-LSTM-Tree\Relation-Classification-using-Bidirectional-LSTM\LCA Shortest Path\checkpoint\modelv1\-440
test accuracy 6.051660516605166


In [63]:
all_predictions[9]

array([17,  8, 11,  0,  3, 11, 11, 12, 12,  7], dtype=int64)

In [48]:
rel2id

{'Other': 0,
 'Entity-Destination(e1,e2)': 1,
 'Cause-Effect(e2,e1)': 2,
 'Member-Collection(e2,e1)': 3,
 'Entity-Origin(e1,e2)': 4,
 'Message-Topic(e1,e2)': 5,
 'Component-Whole(e2,e1)': 6,
 'Component-Whole(e1,e2)': 7,
 'Instrument-Agency(e2,e1)': 8,
 'Product-Producer(e2,e1)': 9,
 'Content-Container(e1,e2)': 10,
 'Cause-Effect(e1,e2)': 11,
 'Product-Producer(e1,e2)': 12,
 'Content-Container(e2,e1)': 13,
 'Entity-Origin(e2,e1)': 14,
 'Message-Topic(e2,e1)': 15,
 'Instrument-Agency(e1,e2)': 16,
 'Member-Collection(e1,e2)': 17,
 'Entity-Destination(e2,e1)': 18}

In [64]:
ID2REL = {v: k for k, v in rel2id.items()}
print(ID2REL)

{0: 'Other', 1: 'Entity-Destination(e1,e2)', 2: 'Cause-Effect(e2,e1)', 3: 'Member-Collection(e2,e1)', 4: 'Entity-Origin(e1,e2)', 5: 'Message-Topic(e1,e2)', 6: 'Component-Whole(e2,e1)', 7: 'Component-Whole(e1,e2)', 8: 'Instrument-Agency(e2,e1)', 9: 'Product-Producer(e2,e1)', 10: 'Content-Container(e1,e2)', 11: 'Cause-Effect(e1,e2)', 12: 'Product-Producer(e1,e2)', 13: 'Content-Container(e2,e1)', 14: 'Entity-Origin(e2,e1)', 15: 'Message-Topic(e2,e1)', 16: 'Instrument-Agency(e1,e2)', 17: 'Member-Collection(e1,e2)', 18: 'Entity-Destination(e2,e1)'}


In [65]:
for pred in batch_predictions:
    print(ID2REL[pred])

Member-Collection(e1,e2)
Instrument-Agency(e2,e1)
Cause-Effect(e1,e2)
Instrument-Agency(e2,e1)
Member-Collection(e2,e1)
Cause-Effect(e1,e2)
Cause-Effect(e1,e2)
Product-Producer(e1,e2)
Product-Producer(e1,e2)
Component-Whole(e1,e2)


In [50]:
sess